# **⚡ Simulated Annealing**

In [ ]:
import numpy as np
from scipy.special import gamma, betaln, betainc, beta, gammaln
from numpy.random import normal, uniform
import math

## **🚥 Parameters**

In [ ]:
# Parameters
H_T = 0.1
g_T = 1.0
T = 5
n = 100

delta1 = T / n
k1 = np.arange(delta1, T + delta1, delta1)
n2 = len(k1)

mu = np.zeros(n2 - 1)
delta2 = 1
k2 = np.arange(1, T + delta2, delta2)
n3 = len(k2)
k4 = np.arange(2, T + delta2, delta2)
k5 = np.arange(1, T, delta2)

b_T = 2 - 2 * H_T
k6 = k4 ** b_T
k7 = k5 ** b_T
k8 = k6 - k7

M_mu = np.zeros(n3 - 1)

M_sim = 100  # number of bootstrap simulations

H_sim = np.zeros(M_sim)
g_sim = np.zeros(M_sim)

## **🧬 Metropolis-Hastings**

In [ ]:
for k in range(M_sim):
    lc2_T = gammaln(1.5 - H_T) - np.log(2) - np.log(H_T) - np.log(b_T) - gammaln(H_T + 0.5) - gammaln(b_T)
    c2_T = np.exp(lc2_T)

    # Draw M_T ~ N(M_mu, sqrt(c2_T * k8))
    M_T = normal(M_mu, np.sqrt(c2_T * k8))

    lk_H1T = np.log(2) + np.log(H_T) + gammaln(1.5 - H_T) + gammaln(0.5 + H_T)
    k_H1T = np.exp(lk_H1T)
    a_T = 1.5 - H_T

    ll_HT = np.log(2) + np.log(H_T) + gammaln(3 - 2 * H_T) + gammaln(0.5 + H_T) - gammaln(1.5 - H_T)
    l_HT = np.exp(ll_HT)

    Q_T = np.zeros(n3)
    dw_T = np.zeros(n3)

    for i1 in range(n3):
        k3 = np.arange(0.001, k2[i1] + delta1, delta1)
        n4 = len(k3)
        Q1_T = np.zeros(n4 - 1)
        for j1 in range(n4 - 1):
            # In R, beta(a_T, a_T) is beta function; here same
            Q1_T[j1] = (l_HT / b_T) * (k_H1T) ** (-1) * g_T * beta(a_T, a_T)

        Q_T[i1] = np.sum(Q1_T)
        dw_T[i1] = b_T * (k2[i1] ** (b_T - 1)) * (l_HT) ** (-1) * delta2

    Z_T = np.zeros(n3 - 1)
    for i2 in range(n3 - 1):
        Z_T[i2] = M_T[i2] + Q_T[i2] * dw_T[i2]

    N = 20000
    H = np.full(N + 2, 0.15)  # +2 to safely index l+1
    g = np.full(N + 2, -1.0)

    L = np.zeros(N + 2)
    L1 = np.zeros(n3 - 1)
    L2 = np.zeros(n3 - 1)
    b_vec = np.zeros(2)
    T_vals = np.zeros(N + 2)
    t_seq = np.arange(2, N + 2)

    for l in range(N):
        k_H1 = 2 * H[l] * gamma(1.5 - H[l]) * gamma(0.5 + H[l])
        l_H = 2 * H[l] * gamma(3 - 2 * H[l]) * gamma(0.5 + H[l]) / gamma(1.5 - H[l])
        a = 1.5 - H[l]
        b = 2 - 2 * H[l]
        c2 = gamma(1.5 - H[l]) / (2 * H[l] * b * gamma(H[l] + 0.5) * gamma(b))

        Q = np.zeros(n3)
        w1 = np.zeros(n3)

        for i1 in range(n3):
            k3 = np.arange(0.001, k2[i1] + delta1, delta1)
            n4 = len(k3)
            Q1 = np.zeros(n4 - 1)
            for j1 in range(n4 - 1):
                Q1[j1] = (l_H / b) * (k_H1) ** (-1) * g[l] * beta(a, a)
            Q[i1] = np.sum(Q1)
            w1[i1] = (k2[i1] ** b) * (l_H) ** (-1)

        k_6 = k4 ** b
        k_7 = k5 ** b
        k_8 = k_6 - k_7
        sd1 = np.sqrt(c2 * k_8)

        M1 = np.zeros(n3 - 1)
        for i2 in range(n3 - 1):
            M1[i2] = Z_T[i2] - Q[i2] * (w1[i2 + 1] - w1[i2])
            L1[i2] = -np.log(sd1[i2]) - (M1[i2] ** 2) / (2 * (sd1[i2] ** 2))

        L[l] = np.sum(L1)

        T_vals[l] = 1 / (np.log(np.log(t_seq[l])))
        if l > 100:
            T_vals[l] = 1 / np.log(np.log(np.log(t_seq[l])))

        u1 = uniform(0, 1, 2)
        for j in range(2):
            b_vec[j] = 1 if u1[j] < 0.5 else -1

        epsilon = normal(0, 1)
        a1 = 0.001
        a2 = 0.0

        H[l + 1] = H[l] + b_vec[0] * a1 * abs(epsilon)
        g[l + 1] = g[l] + b_vec[1] * a2 * abs(epsilon)

        if 0 < H[l + 1] < 1:
            k_H1 = 2 * H[l + 1] * gamma(1.5 - H[l + 1]) * gamma(0.5 + H[l + 1])
            l_H = 2 * H[l + 1] * gamma(3 - 2 * H[l + 1]) * gamma(0.5 + H[l + 1]) / gamma(1.5 - H[l + 1])
            a = 1.5 - H[l + 1]
            b = 2 - 2 * H[l + 1]
            c2 = gamma(1.5 - H[l + 1]) / (2 * H[l + 1] * b * gamma(H[l + 1] + 0.5) * gamma(b))

            Q2 = np.zeros(n3)
            w2 = np.zeros(n3)

            for i1 in range(n3):
                k3 = np.arange(0.001, k2[i1] + delta1, delta1)
                n4 = len(k3)
                Q3 = np.zeros(n4 - 1)
                for j1 in range(n4 - 1):
                    Q3[j1] = (l_H / b) * (k_H1) ** (-1) * g[l + 1] * beta(a, a)
                Q2[i1] = np.sum(Q3)
                w2[i1] = (k2[i1] ** b) * (l_H) ** (-1)

            k_6 = k4 ** b
            k_7 = k5 ** b
            k_8 = k_6 - k_7
            sd2 = np.sqrt(c2 * k_8)

            M2 = np.zeros(n3 - 1)
            for i2 in range(n3 - 1):
                M2[i2] = Z_T[i2] - Q2[i2] * (w2[i2 + 1] - w2[i2])
                L2[i2] = -np.log(sd2[i2]) - (M2[i2] ** 2) / (2 * (sd2[i2] ** 2))

            L[l + 1] = np.sum(L2)
            p = min(0, (L[l + 1] - L[l]) / T_vals[l])
            u2 = uniform(0, 1)

            if np.log(u2) >= p:
                H[l + 1] = H[l]
                g[l + 1] = g[l]
        else:
            H[l + 1] = H[l]
            g[l + 1] = g[l]

    max_L = np.max(L)
    l_star = np.argmax(L)

    H_sim[k] = H[l_star]
    g_sim[k] = g[l_star]

    print(k + 1, H_sim[k], g_sim[k])


## **📒 Saving**

In [ ]:
xboot = np.column_stack((H_sim, g_sim))
np.savetxt("bootstrap_mle2.txt", xboot, fmt="%.6f", delimiter="\t")